In [18]:
import subprocess
import re
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf

from sklearn.linear_model import LinearRegression

from pyopenms import *

In [19]:
class Filter:
    def __init__(self, features):
        """ Filter by regression
        Returns
        -------
        feature  :  feature file or table
        """
        self.features = features
    
            
    def formatFeatures(self, col_pattern='Peak area', meta=None, 
                       norm=True):
        if type(self.features)==str:
            feat_tab = pd.read_csv(self.features)
        else:
            feat_tab = self.features

        feat_area = feat_tab.copy()[feat_tab.columns[feat_tab.columns.str.contains(col_pattern)]]
        if meta is None:
            meta = feat_area.columns.str.replace('.mzML Peak area', '').str.split('_').tolist()
            # assumes DILX dilution factor is on position 3
            meta = [x[1] for x in meta]

        # Normalize samples
        # scale?
        if norm:
            feat_area = feat_area.apply(lambda a: a/a.sum())

        # move features to columns and adds metadata
        feat_area = feat_area.T
        feat_area.reset_index(inplace=True, drop=True)
        feat_area['Groups'] = meta

        # Average technical replicates
        feat_area = feat_area.groupby('Groups').mean()

        # Assumes number describing increasing or decreacing ibdex for dilution
        # here ORI is used for original sample
        y = feat_area.index.str.replace('[^0-9]', '').str.replace('^$', '0').tolist()
        y = np.array([float(i) for i in y]).reshape(-1, 1)

        self.feat_tab = feat_tab
        self.feat_area = feat_area
        self.y = y 

        
    def filterReg(self, feat_idx, r2=0.9, sig='-', 
                  plot=False):
        X = self.feat_area[feat_idx].to_numpy()
        y = self.y
        
        X = X.reshape(-1, 1)
        reg = LinearRegression()
        reg.fit(X, y)
        sc = reg.score(X, y)
        b1 = reg.coef_[0][0]

        if plot:
            plt.scatter(y, X)

            plt.plot(reg.predict(X), X)

            plt.ylabel('Normalized Area')
            plt.xlabel('Dilution')

            plt.xticks([0, 1, 2, 3, 4], ['0', '1', '2', '3', '4'])
            plt.text(y[2,0], X[1,0], r"$R^2=%s$" % np.round(sc, 3))

        if sig=='-':
            if sc > r2 and b1 < 0:
                return 1
            else:
                return 0
        else:
            if sc > r2 and b1 > 0:
                return 1
            else:
                return 0
    
        
    def filterFeatures(self, r2=0.9, sig='-'):
        sel = self.feat_area.apply(lambda a: self.filterReg(a.name, r2=r2, sig=sig))
        
        self.sel = sel
        
        print('Number of features:', len(sel))
        print('Number of features after filtering:', sel.sum())    


In [20]:
samp_010 = Filter('177_G5_ALLNs_MS1.csv')
samp_010.formatFeatures()
samp_010.filterFeatures()

Numeber of fearures: 4163
Numeber of fearures after filtering: 123


In [21]:
samp_010ms2 = Filter('177_G5_ALLNs_MSMS.csv')
samp_010ms2.formatFeatures()
samp_010ms2.filterFeatures()

Numeber of fearures: 4504
Numeber of fearures after filtering: 114


In [22]:
def matchFeats(query, ref, ppm=15, rtabs=20):
    mzdiff = ref['row m/z'].apply(lambda a: ((a-query['row m/z'])/query['row m/z'])*10**6).abs() 
    rtdiff = (ref['row retention time']-query['row retention time']).abs()
    
    diff = (mzdiff < ppm) & (rtdiff < rtabs)
    
    ans = ref.loc[diff, ['row ID', 'row m/z', 'row retention time']]
    
    if len(ans):
        ans['q-id'] = query['row ID']
        ans['q-m/z'] = query['row m/z']
        ans['q-rt'] = query['row retention time']
        ans['ppm'] = mzdiff[diff]
        ans['rtabs'] = rtdiff[diff]
        return ans
    else:
        return pd.DataFrame()

In [23]:
qlist = []

for q in samp_010.sel[samp_010.sel==1].index:
    qlist.append(matchFeats(samp_010.feat_tab.loc[q, ['row ID', 'row m/z', 'row retention time']], samp_010ms2.feat_tab))

In [24]:
qtab = pd.concat(qlist).reset_index()

In [25]:
def mzXML2mzML(filepath):
    subprocess.run(['FileConverter', '-in', filepath, '-out', filepath.replace('mzXML', 'mzML')])

def loadExp(fname):
    exp_in = MSExperiment()
    FileHandler().loadExperiment(fname, exp_in)
    return(exp_in)

def calcTIC(exp):
     tic = 0
     for spec in exp:
         if spec.getMSLevel() == 1:
             mz, i = spec.get_peaks()
             tic += sum(i)
     return tic

def getTIC(exp):
     tic = {} 
     lmz = []
     li = []
     rt = []
     for spec in exp:
         if spec.getMSLevel() == 1:
             mz, i = spec.get_peaks()
             lmz.append(mz)
             li.append(i)
             rt.append(spec.getRT())
     tic['mz'] = lmz
     tic['i'] = li
     tic['rt'] = rt 
     return tic

def getXIC(mass, retention, tic, type='BP', rttol=20, ppm=20, **kwargs):
    xic = {} 
    mztol = mass/((ppm/(10**6))+1.0)
    mzdiff = abs(mztol - mass) 
    rt = np.array(tic['rt']) 
    rtidx = np.where((rt>retention-rttol) & (rt<retention+rttol)) 
    lmz = []
    lrt = []
    li = []
    lxic = []
    for idx in rtidx[0]:
        mztemp = np.array(tic['mz'][idx]) 
        itemp = np.array(tic['i'][idx]) 
        mzidx = np.where((mztemp>mass-mzdiff) & (mztemp<mass+mzdiff)) 
        if len(mzidx[0]):
            lmz += list(mztemp[mzidx])
            li += list(itemp[mzidx])
            lrt.append(rt[idx])
            if type=='TI':
                lxic.append(sum(itemp[mzidx]))
            elif type=='BP':
                if len(itemp[mzidx]):
                    lxic.append(max(itemp[mzidx]))
                else:
                    lxic.append(0)
    xic['mz'] = lmz
    xic['rt'] = lrt
    xic['i'] = li
    xic['xic'] = lxic
    return xic

def getSingleSpectrum(mgffile, scanindex):
    spectrum = {}
    f = open(mgffile)
    lines = np.array(f.readlines())
    f.close()
    bg = np.where(lines=='BEGIN IONS\n')
    ed = np.where(lines=='END IONS\n')
    p = np.where(lines=='SCANS='+str(scanindex)+'\n')
    bgp = bg[0][np.where((bg[0]-p[0])>0)[0][0]-1]
    edp = ed[0][np.where((ed[0]-p[0])>0)[0][0]]+1
    msms1 = []
    for line in lines[bgp:edp]:
        try:
            mz, rt = line.split(' ')
            msms1.append((float(mz), float(rt)))
        except:
            pass
    spectrum[scanindex] = pd.DataFrame(msms1, columns=['Mass', 'Intensity'])
    return(spectrum)


def plotXIC(mz, rt, exp, out, title='', type='XIC', format='png'):
    tic = getTIC(exp)
    xic = getXIC(mz, rt, tic, type='TI')
    if format=='pdf':
        pdf = matplotlib.backends.backend_pdf.PdfPages(out+'.pdf') 
    if type=='BPC':
        fig = plt.figure() 
        if title !='':
            fig.suptitle(title, fontsize=12)
        ax1 = fig.add_subplot(111) 
        ax1.plot(tic['rt'], [max(x) if len(x) else 0 for x in tic['i']], lw=2) 
        if format=='png':
            fig.savefig(out+'.png')
        elif format=='pdf':
            pdf.savefig( fig )   
            pdf.close() 
        plt.close('all') 
    if type=='BPC_XIC':
        fig = plt.figure() 
        if title !='':
            fig.suptitle(title, fontsize=12)
        ax1 = fig.add_subplot(111) 
        ax1.plot(tic['rt'], [max(x) if len(x) else 0 for x in tic['i']], lw=2) 
        ax1.plot(xic['rt'], xic['xic'], lw=2, color='r') 
        if format=='png':
            fig.savefig(out+'.png')
        elif format=='pdf':
            pdf.savefig( fig )   
            pdf.close() 
        plt.close('all') 
    if type=='XIC':
        fig = plt.figure() 
        ax1 = fig.add_subplot(111) 
        fig = plt.figure()
        if title !='':
            fig.suptitle(title, fontsize=12)
        ax1 = fig.add_subplot(111) 
        ax1.plot(xic['rt'], xic['xic'], lw=4, color='r') 
        if format=='png':
            fig.savefig(out+'.png')
        elif fot=='pdf':
            pdf.savefig( fig )   
            pdf.close() 
        plt.close('all') 
    exp.reset()

def overlayXIC(fnames, path, mz, rt, fsz, 
               save=False, out='', **kwargs):
    cm = plt.get_cmap('gist_rainbow')

    cdict = {}
    for i in range(len(fnames)):
        cdict[re.sub('_POS_1-...*_', '_', fnames[i]).split('_')[2]] = i

    for k,v in cdict.items():
        cdict[k] = cm(v/3*3.0/len(cdict))

    for fn in fnames:
        leg = re.sub('_POS_1-...*_', '_', fn)
        cid = leg.split('_')[2]
        try:
            fn.replace('mzXML', 'mzML')
        except:
            print('Not a .mzXML file')
        exp = loadExp(f'{path}/{fn}')
        tic = getTIC(exp)
        xic = getXIC(mz, rt, tic, **kwargs)
        plt.plot(xic['rt'], xic['xic'], color=cdict[cid], label=leg)
        exp.reset()

    plt.legend(fontsize=fsz)
    if save:
        plt.savefig(out)
        plt.show()
    else:
        plt.show()

In [26]:
fnames = samp_010.feat_tab.columns
fnames = fnames[fnames.str.contains('Peak area')].str.replace(' Peak area', '').tolist()

In [27]:
path = './MS1'

for fn in fnames:
    continue

In [28]:
qunique = qtab[qtab.columns[4:7]].drop_duplicates()

In [ ]:
for unique in qunique.index:
    overlayXIC(fnames, path, mz=qunique['q-m/z'][unique], rt=qunique['q-rt'][unique]*60, save=True, out='./images/'+str(qunique['q-id'][unique])+'.png', fsz=10, type='TI', ppm=15)

In [29]:
    import json
    import re
    import os

    import pandas as pd
    import numpy as np
    from IPython.display import SVG
    import matplotlib.pyplot as plt

    html = '''<!DOCTYPE html>
    <html>
      <head>
        <meta http-equiv="content-type" content="text/html; charset=UTF-8">
        <title>NPMINE</title>
        <script src="https://code.jquery.com/jquery-3.3.1.js" integrity="sha256-2Kok7MbOyxpgUVvAk/HJ2jigOSYS2auK4Pfzbm7uH60=" crossorigin="anonymous"></script>
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
        <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>
        <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.6.1/css/all.css" integrity="sha384-gfdkjb5BdAXd+lj+gudLWI+BXq4IuLW5IT+brZEZsLFm++aCMlF1V92rMkPaX4PP" crossorigin="anonymous">
    </head>

      <body>
        <!-- A grey horizontal navbar that becomes vertical on small screens -->
        <nav class="navbar navbar-expand-sm bg-light navbar-light">
            <a class="navbar-brand" href="">
            </a>

          <!-- Links-->
          <ul class="navbar-nav">
            <li class="nav-item">
              <a class="nav-link" href="http://ccbl.fcfrp.usp.br">CCBL</a>
            </li>
          </ul>

        </nav>
        <div id="content">

        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
        <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>
        <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.6.1/css/all.css" integrity="sha384-gfdkjb5BdAXd+lj+gudLWI+BXq4IuLW5IT+brZEZsLFm++aCMlF1V92rMkPaX4PP" crossorigin="anonymous">

        <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>

        <script src="https://cdn.datatables.net/1.10.4/js/jquery.dataTables.min.js"></script>
        <link rel="stylesheet" href="https://cdn.datatables.net/1.10.4/css/jquery.dataTables.min.css">
        <link rel="stylesheet" href="https://cdn.datatables.net/buttons/2.3.6/css/buttons.dataTables.min.css">
        <script type="text/javascript" src="https://code.jquery.com/jquery-3.5.1.js"></script>
        <script type="text/javascript" src="https://cdn.datatables.net/1.13.4/js/jquery.dataTables.min.js"></script>
        <script type="text/javascript" src="https://cdn.datatables.net/buttons/2.3.6/js/dataTables.buttons.min.js"></script>
        <script type="text/javascript" src="https://cdn.datatables.net/buttons/1.7.1/js/buttons.html5.min.js"></script>
        <script type="text/javascript" src="https://cdn.datatables.net/buttons/2.3.6/js/buttons.print.min.js"></script>
        <script type="text/javascript" src="https://cdn.datatables.net/select/1.6.2/js/dataTables.select.min.js"></script>

        <script>
            var dataSet = REPLACE;
            console.log(dataSet);
            $(document).ready(function() {
            $('#resTable').DataTable( {
                dom: 'Bfrtip',
                buttons: [
                    {
                        extend: 'csv',
                        text: 'Export Selected',
                        exportOptions: {
                            modifier: {
                                selected: true
                            }
                        }
                    }
                ],
                data : dataSet,
                // add column definitions to map your json to the table
                "columns": [
                {title: "row ID"},
                {title: "row m/z"},
                {title: "row retention time"},
                {title: "XIC"}
                ],
                select: true
            } );
            });
        </script>

        <div class="m-5">
            <table id="resTable" class="display" style="width:100%" >
            <thead>
            <tr>
            <th>row ID</th>
            <th>row m/z</th>
            <th>row retention time</th>
            <th>XIC</th>
            </tr>
            </thead>
            </table>
            
        </div>
        </div>

      </body>
    </html>'''


    def create_report(report_print, out_file='regfilter_report.html'):
        """Creates an html report from NPMINE's results
        Parameters
        ----------
        report_print: pd.DataFrame
            DataFrame containing columns 'doi', 'pubchem', 'ExactMolWt', 'smiles','source'.
        dois: str or list
            Directory of doi link files or link list.
        out_file: str
            HTML output file name.
        useSVG: bool
            If svg format should be used. Default is png.
        Returns
            Report html file.
        -------
        """

        for i in report_print.index:
            fig = "./images/%s.png" % report_print.loc[i, 'row ID']
            report_print.loc[i, 'XIC'] = '<img src="%s" width="200" height="200">' % fig

        html_local = re.sub('REPLACE', json.dumps(report_print.apply(lambda a: a.tolist(), axis=1).tolist()), html)
        with open(out_file, 'w+') as f:
            f.write(html_local)


In [30]:
#When using html page, select spectra with no linearity 
qunique_html = qunique.rename({'q-id': 'row ID', 'q-m/z': 'row m/z', 'q-rt': 'row retention time'}, axis=1)
create_report(qunique_html)

In [31]:
import pandas as pd
df = pd.read_csv('NPMINE (1).csv')
lista = df.iloc[:, 0].tolist()

In [32]:
dfx = qunique_html.reset_index()
dfx.drop(['index'], axis=1, inplace=True)
dfx.set_index('row ID', inplace = True)
dfx.index = dfx.index.astype(int)
dfx.drop(lista, 0, inplace = True)

name = []
time = []
time_tol = []
mass_begin = []
mass_end = []

for item in dfx.index:
    name.append('')
    time.append(dfx['row retention time'][item]*60)
    time_tol.append(30)
    mass_begin.append(dfx['row m/z'][item])
    mass_end.append('')
    
col_names = ["Compound Name", "Time", "Time Tolerance", "Mass Begin", "Mass End"]
SPL = pd.DataFrame(zip(name,time,time_tol,mass_begin,mass_end), columns = col_names)
SPL.to_csv("SPL_list.csv")
SPL

,Compound Name,Time,Time Tolerance,Mass Begin,Mass End
0,,573.2211,30,121.056395,
1,,1725.9048,30,431.275854,
2,,1620.8975,30,405.263045,
3,,1726.2556,30,355.261855,
4,,874.8222,30,261.126590,
...,...,...,...,...,...
92,,605.3351,30,121.057529,
93,,163.8117,30,268.109361,
94,,1794.4394,30,551.259003,
95,,1373.3057,30,317.211743,
